<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285 - Computación Científica </h1>
    <h2> Laboratorio 5</h2>
    <h2> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> 2025-1</h2>
</center>

## Librerias (No utilice ni importe librerias extras)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from scipy.integrate import solve_ivp

FS = 20
plt.rcParams.update({
    'font.size': FS,
    'axes.labelsize': FS,
    'font.family': 'sans-serif',
    'font.sans-serif': 'DejaVu Sans'
})


## Funciones disponibles

- **np.linalg.norm(x, ord=np.inf)**: Calcula la norma infinito del vector $\mathbf{x}$, que corresponde al valor absoluto máximo entre sus componentes.

- **np.dot(a, b)**: Obtiene el producto interno entre el vector a y b. En caso de que a sea una matriz, entrega el producto matriz-vector respectivo. Para esto es posible usar el operador @.
- **np.{zeros, ones}(n)**: Genera un vector {nulo, de unos} de dimensión $n$.
- **np.{zeros, ones}((n,m))**: Genera una matriz {nula, de unos} de dimensión $n \times m$.
- **np.eyes(k)**: Genera la matriz identidad $I_k$ de dimensión $k \times k$
- **np.transpose(A)**: Obtiene la transpuesta de A, equivalente a A.T
- **np.sqrt(x)**: Calcula la raíz cuadrada elemento a elemento del vector x.
- **np.power(x, p)**: Calcula la potencia a la $p$ elemento a elemento del vector x.
- **np.any(x)**: Retorna Trues si algún elemento del array x tiene valor booleano True
- **np.all(x)**: Retorna Trues si todos los elementos del array x tienen valor booleano True

## Optimización en $\mathbb{R}^n$: Resolviendo un Sistema de Ecuaciones No Lineales

La extensión para resolver **sistema de ecuaciones no lineales** es directa.
Considere la siguiente modificación en la definición de $\mathscr{L}(x_1,x_2)$ particularmente para $\mathbb{R}^2$,
$$
\begin{align*}
    \mathscr{L}(\mathbf{x}) &= \dfrac{1}{2}\left(f_1(\mathbf{x})^2+f_2(\mathbf{x})^2\right),
\end{align*}
$$
o en su forma vectorial,
$$
\begin{align*}
    \mathscr{L}\left(\mathbf{x}\right)  &= \dfrac{1}{2}\left\|\mathbf{F}(\mathbf{x})\right\|^2_2,
\end{align*}
$$
donde $\mathbf{F}(\mathbf{x})=\left[f_1(\mathbf{x}), f_2(\mathbf{x})\right]^{\top}$
Es decir, si encontramos el mínimo de $\mathscr{L}(\mathbf{x})$, encontramos la raíz al sistema de ecuaciones no lineales $\mathbf{F}(\mathbf{x})=\mathbf{0}$.

Entonces, siguiendo el procedimiento presentado en el Contexto del Laboratorio, podemos definir $\mathbf{x}(t)=[x_1(t),x_2(t)]^{\top}$, ahora si derivamos con respecto a $t$ $\mathscr{L}(\mathbf{x}(t))$ y definimos $\dot{x}_1(t)$ y $\dot{x}_2(t)$ tal que la pendiente sea negativa, se obtiene,
$$
\begin{align*}
     \newcommand{\arraystretch}{1.25}
    \begin{bmatrix} \dot{x}_1(t)\\ \dot{x}_2(t) \end{bmatrix}
        &=
        \newcommand{\arraystretch}{2}
        \begin{bmatrix}
            -\dfrac{\partial \mathscr{L}}{\partial x_1}(\mathbf{x})
        \\
            -\dfrac{\partial \mathscr{L}}{\partial x_2}(\mathbf{x})
        \end{bmatrix},
\end{align*}
$$
o vectorialmente,
$$
\begin{align*}
    \dot{\mathbf{x}}
        &= - \nabla \mathscr{L}
\end{align*}
$$
lo cual es un sistema dinámico que podedmos resolver con alguna condición inicial.

## Laboratorio

Considere la siguiente definición de $\mathbf{F}\left(\mathbf{x}\right)$:

$$
\mathbf{F}\left(\mathbf{x}\right) =
\begin{bmatrix}
x_{0}^2 + x_{1} -1 \\  
x_{1}^{2} + \sin\left(x_{1}\,x_{2}\right) -1 \\  
\exp\left(x_{0}\right) x_{2}^2 -1
\end{bmatrix}
$$

y por lo tanto, $\mathscr{L}\left(\mathbf{x}\right)$ se define como:

$$
\mathscr{L}\left(\mathbf{x}\right) = \dfrac{1}{2}
\begin{bmatrix}
\left(x_{0}^2 + x_{1} -1 \right)^{2} + \left( x_{1}^{2} + \sin\left(x_{1}\,x_{2}\right) -1 \right)^{2} + \left( \exp\left(x_{0}\right) x_{2}^2 -1\right)^{2}
\end{bmatrix}
$$

In [1]:
def F(x):
    out = np.zeros_like(x)
    out[0] = np.power(x[0],2.) + x[1] - 1.
    out[1] = np.power(x[1],2.) + np.sin(x[1]*x[2]) - 1.
    out[2] = np.exp(x[0])* np.power(x[2],2.) - 1.
    return out

In [3]:
def L_nonlinear(x):
    return 0.5*np.power(np.linalg.norm(F(x)),2.)

## Pregunta 1 (35 puntos):
Desarrolle la función `RHS(t,x)` la cual calcula $- \nabla \mathscr{L}\left(\mathbf{x}\right)$, es decir el *Right Hand Side* asociado al sistema dinámico, utilizando las derivadas explicitas de $\nabla \mathscr{L}\left(\mathbf{x}\right)$.

In [5]:
def RHS(t,x):
    """
    Input:
    t:      (float) Variable tiempo (añadida por comletitud).
    x:              (ndarray) Vector estado a evaluar.

    Output:
    minus_grad_L:   (ndarray) Gradiente negativo de la función L(x),
    """
    #------ acá va su código ------
    fx = F(x)

    J = np.zeros((3, 3))
    J[0, 0] = 2 * x[0]
    J[0, 1] = 1
    J[0, 2] = 0


    J[1, 0] = 0
    J[1, 1] = 2 * x[1] + x[2] * np.cos(x[1] * x[2])
    J[1, 2] = x[1] * np.cos(x[1] * x[2])


    J[2, 0] = np.exp(x[0]) * x[2]**2
    J[2, 1] = 0
    J[2, 2] = 2 * np.exp(x[0]) * x[2]

    grad_L = np.dot(np.transpose(J),fx)
    minus_grad_L = -grad_L
    #------ acá va su código ------
    return minus_grad_L
x_test = np.array([1.0, 1.0, 1.0])
t_test = 0.0

rhs_val = RHS(t_test, x_test)
print("RHS(t, x) =", rhs_val)


RHS(t, x) = [-6.67077427 -3.13759068 -9.79619725]


## Pregunta 2 (35 puntos):

Construya la función `Find_StationaryState(sol, times, eps, F=RHS, T_max=none)` que permita determinar el instante de tiempo en el cual el sistema dinámico alcanza un **estado estacionario**, utilizando un umbral $\varepsilon$ como tolerancia.

 *Hint: think about the condition & the usage of $\varepsilon$.*

In [10]:
def Find_StationaryState(sol, times, eps, f=RHS, T_max=None):
    """
    Input:
    sol:       (ndarray) Solución x(t) en cada instante.
    times:     (ndarray) Tiempos correspondientes.
    eps:       (float) Tolerancia para derivada cercana a cero.
    F:         (callable) Función del lado derecho del sistema dx/dt = F(t,x).
    T_max:     (float or None) Tiempo máximo hasta el cual buscar. Si es None, se usa todo el tiempo disponible.

    Output:
    tau:      (float) Primer tiempo en que se alcanza el estado estacionario. Retorna -1 si no se alcanza.
    """
    #------ acá va su código ------
    tau = -1

    if T_max is None:
        T_max = times[-1]  # usar todo el tiempo disponible si no se especifica

    for i in range(len(times)):
        t = times[i]
        x = sol[i]

        if t > T_max:
            break

        norma = np.linalg.norm(f(t, x), ord=np.inf)

        if norma < eps:
            tau = t
            break


    #------ acá va su código ------
    return tau


A continuación, puede visualizar la evolución y convergencia del sistema dinámico implementado con el uso de `Solve_ivp`. así como en que tiempo se alcanza el estado estacionario según un umbral $\varepsilon = 10^{-3}$.





In [11]:
####################################
# No modifique, solo ejecute
####################################

def interactive_convergence(x0_0=0.0, x1_0=0.0, x2_0=0.0, h=0.01, max_steps=5000):
    x0 = np.array([x0_0, x1_0, x2_0])
    max_steps = int(max_steps)  # Conversión segura aquí
    t_span = (0, h * max_steps)
    t_eval = np.linspace(t_span[0], t_span[1], max_steps + 1)

    sol_ivp = solve_ivp(RHS, t_span, x0, method='RK45', t_eval=t_eval)
    sol = sol_ivp.y.T
    times = sol_ivp.t

    # Encontrar tiempo estacionario
    eps=1e-3
    t_stat = Find_StationaryState(sol, times, eps, RHS)

    # Calcular valores de L para cada paso para visualizar convergencia
    output_residual = np.array([L_nonlinear(sol[i]) for i in range(len(sol))])

    plt.figure(figsize=(14, 5))

    # Plot variables x1, x2, x3 vs tiempo
    plt.subplot(1, 2, 1)
    plt.plot(times, sol[:,0], label=r'$x_0$')
    plt.plot(times, sol[:,1], label=r'$x_1$')
    plt.plot(times, sol[:,2], label=r'$x_2$')
    plt.xlabel('Tiempo')
    plt. ylim(-5, 5)
    if t_stat != -1:
        plt.axvline(x=t_stat, color='r', linestyle='--', label='Estado estacionario')
    plt.title('Evolución de cada componente')
    plt.legend()
    plt.grid(True)

    # Plot L(x) vs tiempo (escala log para visualizar mejor)
    plt.subplot(1, 2, 2)
    plt.semilogy(times, output_residual, label=r'$\mathscr{L}(\mathbf{x}(t))$')
    if t_stat != -1:
        plt.axvline(x=t_stat, color='r', linestyle='--', label='Estado estacionario')
    plt.xlabel('Tiempo')
    plt.title('Convergencia de la función objetivo')
    plt.legend()
    plt.ylim(1e-10, 10)
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Widgets para x0, x1, x2
range_x0 = FloatSlider(min=-3, max=3, step=0.01, value=0, description='x_0')
range_x1 = FloatSlider(min=-3, max=3, step=0.01, value=0, description='x_1')
range_x2 = FloatSlider(min=-3, max=3, step=0.01, value=0, description='x_2')
range_Tmax = FloatSlider(min=10, max=1000, step=1, value=10, description='Max Steps')
range_h = FloatSlider(min=0.001, max=0.1, step=0.01, value=0.05, description='Step Size')

interact(interactive_convergence, x0_0=range_x0, x1_0=range_x1, x2_0=range_x2, max_steps=range_Tmax, h=range_h,)

interactive(children=(FloatSlider(value=0.0, description='x_0', max=3.0, min=-3.0, step=0.01), FloatSlider(val…

<function __main__.interactive_convergence(x0_0=0.0, x1_0=0.0, x2_0=0.0, h=0.01, max_steps=5000)>

## Pregunta 3 (15 puntos):
Apoyandose del gráfico anterior y de lo mencionado anteriormente, explique por qué es posible que se alcance un estado estacionario del sistema dinámico, sin que la función objetivo converga a $0$.

*Hint: Which conditions define a stationary state? and what is the objective function?*

**Se sugiere encarecidamente modificar el *initial guess* del vector de estado $\mathbf{x}$ usando los widgets $x_0$, $x_1$ y $x_2$, así como los valores Step Size $(h)$ y Max Steps. Cada uno de estos valores influye en la convergencia y comportamiento del sistema, por lo cual es importante para su análisis.**

---
### Pregunta 3

Un estado estacionario se alcanza cuando la derivada temporal del sistema dinámico se anula, es decir, cuando:

$$
\dot{\mathbf{x}}(t) = \mathbf{0}
$$

Esto implica que el sistema deja de evolucionar con el tiempo, ya que sus componentes se estabilizan.

Por otro lado, la función objetivo $$\mathscr{L}(\mathbf{x}(t))$$ representa una medida del error o residuo en la solución. En un caso ideal, se esperaría que esta converja a cero si el sistema encuentra una solución óptima.

Sin embargo, es posible que el sistema alcance un estado estacionario sin que $$ \mathscr{L}(\mathbf{x}(t)) \to 0 $$. Esto se debe a que:

- El sistema puede detenerse en un punto de equilibrio que no corresponde al mínimo global de $ \mathscr{L} $, sino a un mínimo local o un punto donde las fuerzas se compensan.
- La función objetivo puede estabilizarse en un valor distinto de cero si el estado alcanzado no satisface por completo las condiciones ideales.

Para comprobar esto, se utilizaron los siguientes parámetros:

- $$ x_0 = 1.04 $$ $$ x_1 = 0.65 $$ $$ x_2 = -0.57 $$
- Step size $$ h = 0.06 $$
- Max Steps = 550

Con estos valores, se observó que el sistema alcanzó un estado estacionario (línea roja en el gráfico), pero la función objetivo $\mathscr{L} $ se estabilizó en un valor cercano a $ 10^{-2} $, sin converger a cero.

Esto demuestra que la condición de estacionariedad (derivada nula) no implica necesariamente la minimización de la función objetivo, lo cual depende también del estado inicial y de los parámetros de integración numérica como `step size` y `max steps`.



---

## Pregunta 4 (15 puntos):
Considere que el sistema de ecuaciones no lineales es **bajo-determinado**, es decir, el número de ecuaciones $m$ es menor que el número de incógnitas $n$. Por ejemplo:

$$
\mathscr{L}(\mathbf{x}) = \left\| \mathbf{F}(\mathbf{x}) \right\|^2_2 = \left\| \mathbf{b} - A \, \mathbf{x} \right\|^2_2
$$

Con:  
$$
\mathbf{b} \in \mathbb{R}^2 \quad \text{y} \quad A \in \mathbb{R}^{2 \times 3}.
$$

Explique qué implica esta configuración respecto a la existencia y unicidad de las soluciones del sistema, y qué solución entrega el enfoque dinámico utilizado en este laboratorio. Considere los siguientes valores junto con el codigo para fundamentar su respuesta:
$$
A =
\begin{bmatrix}
1 & 2 & 0 \\
0 & 1 & 1
\end{bmatrix},
\quad
\mathbf{b} =
\begin{bmatrix}
b_0 \\
b_1
\end{bmatrix},
\quad
\mathbf{x} =
\begin{bmatrix}
x_0(t) \\
x_1(t) \\
x_2(t)
\end{bmatrix}
$$

**Se deben usar de los siguientes gráficos interactivos para el análisis, donde se puede modificar el *initial guess* del vector de estado $\mathbf{x}$ usando los widgets $x_0$, $x_1$ y $x_2$, así como los valores $b_0$ y $b_1$. Cada uno de estos valores influye en la convergencia y comportamiento del sistema.**

In [12]:
def F_linear(t, x, A, b):
    return -A.T @ (A @ x - b)

def simulate_dynamics(A, b, x0, T=10, h=0.01):
    t_span = (0, T)
    t_eval = np.arange(0, T + h, h)
    sol = solve_ivp(F_linear, t_span, x0, args=(A, b), t_eval=t_eval)
    return sol.t, sol.y.T

def interactive_bajo_determinado(x0_0=0.0, x0_1=0.0, x0_2=0.0, b0=1.0, b1=2.0):
    A = np.array([[1, 2, 0],
                  [0, 1, 1]])
    b = np.array([b0, b1])
    x0 = np.array([x0_0, x0_1, x0_2])

    times, sol = simulate_dynamics(A, b, x0)
    residual = np.linalg.norm(A @ sol.T - b[:, None], axis=0)
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(times, sol[:, 0], label='$x_0$')
    plt.plot(times, sol[:, 1], label='$x_1$')
    plt.plot(times, sol[:, 2], label='$x_2$')
    plt.xlabel('Tiempo')
    plt.ylabel('Componentes de $\\mathbf{x}(t)$')
    plt.title('Evolución de la solución')
    plt.grid(True)
    plt.legend()
    plt.ylim([-10, 10])
    plt.subplot(1, 2, 2)
    plt.semilogy(times, residual, label='$\\|A\\mathbf{x}(t) - \\mathbf{b}\\|$')
    plt.xlabel('Tiempo')
    plt.title('Convergencia del residuo')
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()

interact(interactive_bajo_determinado,
         x0_0=FloatSlider(min=-5, max=5, step=0.1, value=0, description='x_0'),
         x0_1=FloatSlider(min=-5, max=5, step=0.1, value=0, description='x_1'),
         x0_2=FloatSlider(min=-5, max=5, step=0.1, value=0, description='x_2'),
         b0=FloatSlider(min=-5, max=5, step=0.1, value=1, description='b_0'),
         b1=FloatSlider(min=-5, max=5, step=0.1, value=2, description='b_1'));


interactive(children=(FloatSlider(value=0.0, description='x_0', max=5.0, min=-5.0), FloatSlider(value=0.0, des…

---
En un sistema bajo-determinado, como el propuesto donde $A \in \mathbb{R}^{2 \times 3}$ y $\mathbf{b} \in \mathbb{R}^2$, hay menos ecuaciones ($m=2$) que incógnitas ($n=3$). Esto implica que el sistema $\mathbf{F}(\mathbf{x}) = \mathbf{b} - A\mathbf{x}$ tiene infinitas soluciones, ya que el conjunto solución forma un espacio afín de dimensión $n - \text{rank}(A)$.

Por lo tanto, no existe una única solución que satisfaga exactamente $A\mathbf{x} = \mathbf{b}$, sino una familia de soluciones que cumplen la condición. Entre todas estas soluciones, el enfoque dinámico implementado en este laboratorio esta basado en minimizar la función objetivo cuadrática
$$
\mathscr{L}(\mathbf{x}) = \| \mathbf{F}(\mathbf{x}) \|^2_2 = \| \mathbf{b} - A\mathbf{x} \|^2_2
$$
que busca encontrar una solución particular: aquella que minimiza la norma del residuo, es decir, una $\mathbf{x}^*$ tal que:
$$
A \mathbf{x}^* \approx \mathbf{b}
$$
si el sistema es consistente o encuentra una aproximación adecuada si no lo es.

---

### Observaciones con valores específicos

Se realizaron simulaciones usando los siguientes parámetros:

- Estado inicial:
  - $x_0 = 1.10$
  - $x_1 = -1.70$
  - $x_2 = 1.40$
- Parámetros del sistema:
  - $b_0 = 0.10$
  - $b_1 = 0.30$

---
Aca se observa que el sistema converge a un estado estacionario donde las componentes de $\mathbf{x}(t)$ se estabilizan. En la gráfica de la derecha, el residuo $\| A \mathbf{x}(t) - \mathbf{b} \|$ disminuye, pero no tiende exactamente a cero, lo que indica que el sistema converge a unasolución aproximada que minimiza el error, pero no necesariamente cumple exactamente con $A \mathbf{x} = \mathbf{b}$.